# Library

In [1]:
#Library
import sys
import os
import os.path as pth

#!pip install torchplot
import torch
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import torch.backends.cudnn as cudnn
import torchplot as plt



import nets
import datasets
import tools
import layers as L
import train

from io import BytesIO
from datetime import datetime
from pytz import timezone
from slacker import Slacker
from quantization import *


import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import random
import itertools
import math

from tqdm import tqdm
from numba import jit

import warnings
warnings.simplefilter("ignore")



# Functions

In [2]:
# Custom functions - Completed

@jit
def amac(x, w):
    return (x * w).sum()


@jit
def relu(x):
    return np.maximum(0,x)


@jit
def padding(a):
    result = np.zeros((a.shape[0],a.shape[1]+2,a.shape[2]+2))
    for i in range(a.shape[0]):
        result[i] = np.pad(a[i],1)
    return result


@jit(nopython=True)
def maxpooling(x_original):
   
    depth = x_original.shape[0] 
    row = int((x_original.shape[1])/2)
    col = int((x_original.shape[2])/2)

    one_layer = np.zeros((depth,row,col))
    
    for d in range(depth):
        for i in range(row):
            for j in range(col):
                r = x_original[d,2*i:2*i+2,2*j:2*j+2].max()
                one_layer[d,i,j] = r
    return one_layer

get_bin = lambda x, n: format(x, 'b').zfill(n).replace("-","1")


# Partial sum function with point parameters - Completed

# This is partial sum function for convolution layer -> Matches with verilog
@jit(cache=True)
def partial_sum_fa_conv_point(original, bit=5, point = 1):
    a = original
    bit = bit - 2  + (point - 1)
    value = 1.875/(2**(point-1))
    
    result = np.zeros(a.shape[0])
    for d in range(a.shape[0]):
        partial = a[d].sum()
        if partial > value:
            partial = value
        elif partial < -value:
            partial = -value
        else:
            partial = partial
        result[d] = math.trunc(partial* (2**bit)) / (2**bit)
    return result.sum()




# This function matches with our verilog model & reasonable accuracy
@jit(cache=True)
def partial_sum_fa_fc_point(original, bit=5, point=1):
    a = original
    bit = bit - 2  + (point - 1)
    value = 1.875/(2**(point-1))
    
    partial = a.sum()
    if partial > value:
        partial = value
    elif partial < -value:
        partial = -value
    else:
        partial = partial
    result= math.trunc(partial* (2**bit)) / (2**bit)
    return result

In [3]:
# Functions for different fixed points -> Original functions
@jit(nopython=True)
def quant_signed_15_np(original, bit=5):
    bit = bit -2
    original = np.clip(original, -1.875, 1.875)
    original = original * (2**bit)
    
    (row, col) = original.shape
    result = np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            result[i,j] = math.trunc(original[i,j])/ (2**bit)
    return result

@jit
def quant_signed_15_np_fc(original, bit=5):
    bit = bit -2
    original = np.clip(original, -1.875, 1.875)
    original = original * (2**bit)
    
    result = math.trunc(original)/ (2**bit)
    return result


# FC & Conv function with point parameters
# Problem of these functions is that they have different fixed points with bias
# And it does not match with our verilog model
@jit(cache=True)
def fc_fa(x_original, w, b, p=1):
   
    filt = w.shape[0]
    stage = int(x_original.shape[1]/8)
    c = np.zeros((1,filt))
    for f in range(filt):
        re = 0
        for i in range(stage):
            r = x_original[0,i*8:i*8+8] * w[f,i*8:i*8+8]
            re = re + partial_sum_fa_fc_point(r, point=p)
        c[0,f] = quant_signed_15_np_fc(re + b[f])
    return c

@jit(cache=True)
def conv_custom_fa(x_original, w, b, p=1):
   
    filt = w.shape[0]
    depth = x_original.shape[0] 
    row = x_original.shape[1] - 2
    col = x_original.shape[2] - 2

    c = np.zeros((filt,row,col))
    one_layer = np.zeros((row,col))
    
    for f in range(filt):
        for i in range(row):
            for j in range(col):
                r = x_original[:,i:i+3,j:j+3] * w[f]
                one_layer[i,j] = partial_sum_fa_conv_point(r, point=p)
        c[f,:,:] = quant_signed_15_np(one_layer + b[f])
    return c



In [4]:
@jit(nopython=True)
def quant_signed_05_np(original, bit=5):
    bit = bit -1
    original = np.clip(original, -0.9375, 0.9375)
    original = original * (2**bit)
    
    (row, col) = original.shape
    result = np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            result[i,j] = math.trunc(original[i,j])/ (2**bit)
    return result

@jit
def quant_signed_05_np_fc(original, bit=5):
    bit = bit -1
    original = np.clip(original, -0.9375, 0.9375)
    original = original * (2**bit)
    
    result = math.trunc(original)/ (2**bit)
    return result


# FC & Conv function with point parameters - completed
# These functions have same fixed point with bias & match with verilog file
@jit(cache=True)
def fc_fa_05(x_original, w, b, p=1):
   
    filt = w.shape[0]
    stage = int(x_original.shape[1]/8)
    c = np.zeros((1,filt))
    for f in range(filt):
        re = 0
        for i in range(stage):
            r = x_original[0,i*8:i*8+8] * w[f,i*8:i*8+8]
            re = re + partial_sum_fa_fc_point(r, point=p)
        c[0,f] = quant_signed_05_np_fc(re + b[f])
    return c

@jit(cache=True)
def conv_custom_fa_05(x_original, w, b, p=1):
   
    filt = w.shape[0]
    depth = x_original.shape[0] 
    row = x_original.shape[1] - 2
    col = x_original.shape[2] - 2

    c = np.zeros((filt,row,col))
    one_layer = np.zeros((row,col))
    
    for f in range(filt):
        for i in range(row):
            for j in range(col):
                r = x_original[:,i:i+3,j:j+3] * w[f]
                one_layer[i,j] = partial_sum_fa_conv_point(r, point=p)
        c[f,:,:] = quant_signed_05_np(one_layer + b[f])
    return c

# Plot

In [5]:
# Load pretrained model and data

file = open('latest.txt' , 'r' )
line = file.readline()
pretrained_checkpoint = line
file.close()

# Default settings for arch, dataset, and checkpoint
arch = "CNN_627_large"
dataset = "cifar10"
batch_size = 256
pretrained_checkpoint = pretrained_checkpoint

trainloader, _, testloader = datasets.get_mnist(batch_size)
model = nets.mnist_quant()

pretrained_ckpt = torch.load(pretrained_checkpoint)
model.load_state_dict(pretrained_ckpt['state_dict'])
print("########## Loaded checkpoint '{}'".format(pretrained_checkpoint))

print("mnist dataset")
with open("./data_quantized/quant_test_data_mnist.pkl","rb") as f:
    data_list = pickle.load(f)
with open("./data_quantized/quant_test_label_mnist.pkl","rb") as g:
    label_list = pickle.load(g)
    
    
# Load Weights and biases
w1 = model.conv1.weight
b1 = model.conv1.bias

w2 = model.conv2.weight
b2 = model.conv2.bias

w3 = model.conv3.weight
b3 = model.conv3.bias

w4 = model.conv4.weight
b4 = model.conv4.bias

w5 = model.fc5.weight
b5 = model.fc5.bias

w6 = model.fc6.weight
b6 = model.fc6.bias


w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()

Loading MNIST data ... 
Quant MNIST
########## Loaded checkpoint './checkpoints_quant/mnist_quant_mnist/2_18_Time_16_36/checkpoint_7_98.3.tar'
mnist dataset


In [7]:
i = 0
y = label_list[i]
x_original = data_list[i].view(1,32,32).numpy()

a, b, c = conv_custom_fa_plot(x_original,w1,b1)

NameError: name 'conv_custom_fa_plot' is not defined

In [6]:
plt.scatter(range(len(b.flatten())),b.flatten())

NameError: name 'b' is not defined

In [ ]:
plt.plot(b.flatten())

# Open model and file

## Save the data with different fixed point

In [8]:
# MNIST
batch_size=256
distributed=None
trainsampler = None
workers=2

print("Loading MNIST data ... ")
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./MNIST', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=(trainsampler is None), 
              num_workers=workers, pin_memory=True, sampler=trainsampler)

val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])
testset = torchvision.datasets.MNIST(root='./MNIST', train=False, download=True, transform=val_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=False, num_workers=workers, pin_memory=True)

# Saving Data

# Padding
m = nn.ConstantPad2d(2, 0)


# Save train data
data_train = trainset.train_data
label_train = trainset.train_labels

data_train = (data_train.int()-128)/128
data_train = quant_signed_15(data_train)
data_train = m(data_train)

with open("./data_quantized/quant_data_mnist.pkl","wb") as f:
    pickle.dump(data_train, f)
with open("./data_quantized/quant_label_mnist.pkl","wb") as g:
    pickle.dump(label_train, g)
    
    
# Save test data
label = testset.test_labels
data = testset.test_data

data = (data.int()-128)/128
data = quant_signed_15(data)
data = m(data)

with open("./data_quantized/quant_test_data_mnist.pkl","wb") as f:
    pickle.dump(data, f)
with open("./data_quantized/quant_test_label_mnist.pkl","wb") as g:
    pickle.dump(label, g)


Loading MNIST data ... 


## Save the data with same fixed point

In [9]:
# MNIST
batch_size=256
distributed=None
trainsampler = None
workers=2

print("Loading MNIST data ... ")
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./MNIST', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=(trainsampler is None), 
              num_workers=workers, pin_memory=True, sampler=trainsampler)

val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])
testset = torchvision.datasets.MNIST(root='./MNIST', train=False, download=True, transform=val_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=False, num_workers=workers, pin_memory=True)

# Saving Data

# Padding
m = nn.ConstantPad2d(2, 0)


# Save train data
data_train = trainset.train_data
label_train = trainset.train_labels

data_train = (data_train.int()-128)/128
data_train = quant_signed_05(data_train)
data_train = m(data_train)

with open("./data_quantized/quant_data_mnist_05.pkl","wb") as f:
    pickle.dump(data_train, f)
with open("./data_quantized/quant_label_mnist_05.pkl","wb") as g:
    pickle.dump(label_train, g)
    
    
# Save test data
label = testset.test_labels
data = testset.test_data

data = (data.int()-128)/128
data = quant_signed_05(data)
data = m(data)

with open("./data_quantized/quant_test_data_mnist_05.pkl","wb") as f:
    pickle.dump(data, f)
with open("./data_quantized/quant_test_label_mnist_05.pkl","wb") as g:
    pickle.dump(label, g)


Loading MNIST data ... 


# Load the model & input data

In [10]:
# Load pretrained model and data

file = open('latest.txt' , 'r' )
line = file.readline()
pretrained_checkpoint = line
file.close()

# Default settings for arch, dataset, and checkpoint
arch = "CNN_627_large"
dataset = "cifar10"
batch_size = 256
pretrained_checkpoint = pretrained_checkpoint

# trainloader, _, testloader = datasets.get_mnist(batch_size)
model = nets.mnist_quant()

pretrained_ckpt = torch.load(pretrained_checkpoint)
model.load_state_dict(pretrained_ckpt['state_dict'])
print("########## Loaded checkpoint '{}'".format(pretrained_checkpoint))

print("mnist dataset with different fixed point")
with open("./data_quantized/quant_test_data_mnist.pkl","rb") as f:
    data_list = pickle.load(f)
with open("./data_quantized/quant_test_label_mnist.pkl","rb") as g:
    label_list = pickle.load(g)
    

#print("mnist dataset with same fixed point")
#with open("./data_quantized/quant_test_data_mnist_05.pkl","rb") as f:
#    data_list2 = pickle.load(f)
#with open("./data_quantized/quant_test_label_mnist_05.pkl","rb") as g:
#    label_list2 = pickle.load(g)
    
# Load Weights and biases

w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()

Quant MNIST
########## Loaded checkpoint './checkpoints_quant/mnist_quant_mnist/2_18_Time_16_36/checkpoint_7_98.3.tar'
mnist dataset with different fixed point


# Save weight

## Save weight of first conv

In [ ]:
# check there is a path directory, and  if not make it

In [ ]:
def save_conv_weight(weight, bias, file_name, path):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        weight_name = file_name + "_weight.txt"
        bias_name = file_name + "_bias.txt"
    file = open(path + "/" + weight_name,'w')

    a = weight
    a = a * 16
    for d in range(a.shape[1]):
        for f in range(a.shape[0]):
            for i in range(3):
                for j in range(3):
                    file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
    file.close()

    file = open(path + "/" + bias_name,'w')
    a = bias
    a = a * 16
    for i in range(a.shape[0]):
        file.write(get_bin(int(a[i]),5)+"\n")
    file.close()
    print(f"Save {file_name} weight to {path}/{weight_name}\nSave {file_name} bias   to {path}/{bias_name}")

def save_fc_weight(weight, bias, file_name, path, include_bias = 0):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        file_name = file_name + ".txt"
    
    file = open(path + "/" + file_name,'w')

    weight = weight*16
    bias = bias * 16
    for w in range(0, weight.shape[0], 32):
        for mac in range(min(weight.shape[0] - w, 32)):
            if include_bias:
                file.write(get_bin(int(bias[w+mac]),5)+"\n")
            else:
                file.write("0\n")
        for z in range(0, weight.shape[1], 8):
            for mac in range(min(weight.shape[0] - w, 32)):
                for port in range(8):
                    file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")
                    
    file.close()
    print(f"Save {file_name[:-4]} weight to {path}/{file_name}")

In [ ]:
path = "weight_new"

save_conv_weight(w1, b1, "conv1", path)
save_conv_weight(w2, b2, "conv2", path)
save_conv_weight(w3, b3, "conv3", path)
save_conv_weight(w4, b4, "conv4", path)
save_fc_weight(w5, b5, "fc5", path)
save_fc_weight(w6, b6, "fc6", path)

In [ ]:
file = open('./data_extracted/conv1_weight_0311.txt','w')

a = w1
a = a * 16
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),10)+"\n")
file.close()

In [ ]:
file = open('./data_extracted/conv1_weight_0311_bias.txt','w')

a = w1
a = a * 32
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),10)+"\n")
file.close()

## Save weight of second conv

In [ ]:
file = open('./data_extracted/conv2_weight_0311.txt','w')

a = w2
a = a * 16
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
file.close()

## Save weight of thrid conv

In [ ]:
file = open('./data_extracted/conv3_weight_0311.txt','w')

a = w3
a = a * 16
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
file.close()

## Save weight of fourth conv

In [ ]:
file = open('./data_extracted/conv4_weight_0311.txt','w')

a = w4
a = a * 16
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
file.close()

## Save weight of fifth fc

In [ ]:
print(w5.shape)

In [ ]:
file = open('./data_extracted/fc5_weight_0311.txt','w')

a = w5
a = a * 16
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
file.close()

In [ ]:
file = open('./data_extracted/fc5_weight_0311.txt','w')

weight = w5*16

for w in range(0, weight.shape[0], 32):
    for mac in range(min(weight.shape[0] - w, 32)):
#             file.write(get_bin(int(bias[w+mac]),5)+"\n")
        file.write("0\n")
    for z in range(0, weight.shape[1], 8):
        for mac in range(min(weight.shape[0] - w, 32)):
            for port in range(8):
                file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")
                
file.close()

## Save weight of sixth fc

In [ ]:
print(w6.shape)

In [ ]:
file = open('./data_extracted/fc6_weight_0311.txt','w')

a = w6
a = a * 16
for d in range(a.shape[1]):
    for f in range(a.shape[0]):
        for i in range(3):
            for j in range(3):
                file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
file.close()

In [ ]:
file = open('./data_extracted/fc6_weight_0311.txt','w')

weight = w6*16

for w in range(0, weight.shape[0], 32):
    for mac in range(min(weight.shape[0] - w, 32)):
#             file.write(get_bin(int(bias[w+mac]),5)+"\n")
        file.write("0\n")
    for z in range(0, weight.shape[1], 8):
        for mac in range(min(weight.shape[0] - w, 32)):
            for port in range(8):
                file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")
                
file.close()

# Save bias

In [ ]:
b1.shape
file = open('./data_extracted/conv1_bias_0311.txt','w')

a = b1
a = a * 16
for i in range(a.shape[0]):
    file.write(get_bin(int(a[i]),5)+"\n")
file.close()

# Test

## Input

In [13]:
i = 0
y = label_list[i]
x_original = data_list[i].view(1,32,32).numpy()
x_original[0][10] * 8

array([ 0.,  0., -8., -8., -8., -8., -8., -8.,  5.,  7.,  7.,  7.,  7.,
        7.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  2., -4., -8., -8.,
       -8., -8., -8., -8.,  0.,  0.], dtype=float32)

In [12]:
i = 0
y = label_list[i]
x_original = data_list2[i].view(1,32,32).numpy()
x_original[0][10] * 16

NameError: name 'data_list2' is not defined

In [14]:
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

x = x_original
x = x*8

tensor(1024)


In [16]:
x[0][3]

tensor([ 0.,  0., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8.,
        -8., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8., -8.,
        -8., -8.,  0.,  0.])

In [ ]:
get_bin(int(x[d,i,j]),5)

## Input save

In [ ]:
file = open('./data_extracted/conv1_input_0317.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

x = x_original
x = x*8
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

## First conv result save

In [ ]:
b1 = np.zeros(32)
x = x_original
x = padding(x)
#x = conv_custom_fa_wo_quant(x,w1,b1)
x = conv_custom_fa(x,w1,b1, p=1)


file = open('./data_extracted/conv1_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

## First relu&pool result save

In [ ]:
x = relu(maxpooling(x))

file = open('./data_extracted/pool_relu1_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

## Second conv result save

In [ ]:
print(x.shape)
b2 = np.zeros(64)
x = padding(x)
x = conv_custom_fa(x,w2,b2, p=1)

file = open('./data_extracted/conv2_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

## Second relu & pool result save

In [ ]:
x = relu(maxpooling(x))

file = open('./data_extracted/pool_relu2_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

## Thrid conv result save

In [ ]:
print(x.shape)
b3 = np.zeros(128)
x = padding(x)
x = conv_custom_fa(x,w3,b3, p=1)

file = open('./data_extracted/conv3_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

## Thrid pool & relu result save

In [ ]:
x = relu(maxpooling(x))

file = open('./data_extracted/pool_relu3_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

## Fourth conv result save

In [ ]:
print(x.shape)
b4 = np.zeros(256)
x = padding(x)
x = conv_custom_fa(x,w4,b4, p=1)

file = open('./data_extracted/conv4_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

## Fourth pool & relu result save

In [ ]:
x = relu(maxpooling(x))

file = open('./data_extracted/pool_relu4_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

## Fifth fc result save

In [ ]:
print(x.shape)
b5 = np.zeros(32)

x = fc_fa(x,w5,b5, p=1)

file = open('./data_extracted/fc5_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

## Fifth relu result save

In [ ]:
x = relu(x)

file = open('./data_extracted/relu5_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

## Sixth fc result save

In [ ]:
print(x.shape)
b6 = np.zeros(10)

x = fc_fa(x,w6,b6, p=1)

file = open('./data_extracted/fc6_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

## Sixth relu result save

In [ ]:
x = relu(x)

file = open('./data_extracted/relu6_output_0317.txt','w')

save = x*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

In [ ]:
x.shape

# Need to organize

In [ ]:
plt.hist(x.flatten())

In [ ]:
x = maxpooling(x)
x = relu(x)
print(x.shape)
plt.hist(x.flatten())

In [ ]:
print(x.shape)
b3 = np.zeros(128)
x = padding(x)
x, mid = conv_custom_fa_plot(x,w3,b3)

In [ ]:
x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa_test(x,w4,b4)
x = maxpooling(x)
x = relu(x)

In [ ]:
print("mnist dataset")
with open("./data_quantized/quant_data_mnist.pkl","rb") as f:
    data_list = pickle.load(f)
with open("./data_quantized/quant_label_mnist.pkl","rb") as g:
    label_list = pickle.load(g)

In [ ]:
file = open('./data_extracted/input.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_15(x_original)).sum())
x_original = quant_signed_15(x_original)

x = x_original
x = x*8
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

In [ ]:
print("--------------shape----------------")
print(x_original.shape)

x = x_original.numpy()
x = padding(x)

result_original = conv_custom_fa(x, w1, b1)
result = result_original*8


print(result_original.shape)
print(sum(sum(sum(result >= 16))))

file = open('./data_extracted/output/conv1_output.txt','w')

x = result
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

In [ ]:
x = maxpooling(x)
x = relu(x)

In [ ]:
file = open('./data_extracted/output/acc1__custom_output.txt','w')

save = mid*8
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

In [ ]:
x = x_original
print(x.shape)
b1 = np.zeros(32)
x = padding(x)
x, mid, before_sum = conv_custom_fa_plot(x,w1,b1)
plt.hist(x.flatten())

In [ ]:
plt.hist(mid.flatten())

# Test One Shot

In [ ]:
# Load pretrained model and data

file = open('latest.txt' , 'r' )
line = file.readline()
pretrained_checkpoint = line
file.close()

# Default settings for arch, dataset, and checkpoint
arch = "CNN_627_large"
dataset = "cifar10"
batch_size = 256
pretrained_checkpoint = pretrained_checkpoint

trainloader, _, testloader = datasets.get_mnist(batch_size)
model = nets.mnist_quant()

pretrained_ckpt = torch.load(pretrained_checkpoint)
model.load_state_dict(pretrained_ckpt['state_dict'])
print("########## Loaded checkpoint '{}'".format(pretrained_checkpoint))

print("mnist dataset")
with open("./data_quantized/quant_test_data_mnist.pkl","rb") as f:
    data_list = pickle.load(f)
with open("./data_quantized/quant_test_label_mnist.pkl","rb") as g:
    label_list = pickle.load(g)
    
    
# Load Weights and biases
w1 = model.conv1.weight
b1 = model.conv1.bias

w2 = model.conv2.weight
b2 = model.conv2.bias

w3 = model.conv3.weight
b3 = model.conv3.bias

w4 = model.conv4.weight
b4 = model.conv4.bias

w5 = model.fc5.weight
b5 = model.fc5.bias

w6 = model.fc6.weight
b6 = model.fc6.bias


w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()

## Custom model

## With different fixed points

In [5]:
# Evaluate with Custom model
correct = 0
for i in range(256):

    y = label_list[i]

    x = data_list[i].view(1,32,32).numpy()

    x = padding(x)
    x = conv_custom_fa(x,w1,b1, p=1)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa(x,w2,b2, p=2)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa(x,w3,b3, p=3)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa(x,w4,b4, p=4)
    x = maxpooling(x)
    x = relu(x)

    x = x.reshape(1,1024)

    x = fc_fa(x,w5,b5, p=3)
    x = relu(x)

    x = fc_fa(x,w6,b6, p=1)


    if (np.argmax(x) == y.item()):
        correct += 1

print(correct/(i+1))

NameError: name 'conv_custom_fa' is not defined

## With same fixed point

In [6]:
# Evaluate with Custom model
correct = 0
#randlist = random.sample(range(0, 10000), 256)
randlist = [i for i in range(256)]
for i in randlist:

    y = label_list[i]
    x = data_list[i].view(1,32,32).numpy()

    x = padding(x)
    x = conv_custom_fa_05(x,w1,b1, p=1)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa_05(x,w2,b2, p=2)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa_05(x,w3,b3, p=3)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa_05(x,w4,b4, p=4)
    x = maxpooling(x)
    x = relu(x)

    x = x.reshape(1,1024)

    x = fc_fa_05(x,w5,b5, p=3)
    x = relu(x)

    x = fc_fa_05(x,w6,b6, p=1)


    if (np.argmax(x) == y.item()):
        correct += 1

print(correct/len(randlist))

0.83203125


In [5]:
i = 0
y = label_list[i]
# print(y)
x = data_list[i].view(1,32,32).numpy()

x = padding(x)
# print(x[0][10])
x = conv_custom_fa_05(x,w1,b1, p=1)
# print(x[0])
# x = maxpooling(x)
# x = relu(x)

# x = padding(x)
# x = conv_custom_fa_05(x,w2,b2, p=2)
# x = maxpooling(x)
# x = relu(x)

print(x[10])
# x = padding(x)
# x = conv_custom_fa_05(x,w3,b3, p=3)
# x = maxpooling(x)
# x = relu(x)

# x = padding(x)
# x = conv_custom_fa_05(x,w4,b4, p=4)
# x = maxpooling(x)
# x = relu(x)

# x = x.reshape(1,1024)
# for i in range(1024):
#     print(x[0][i])

[[-0.5625 -0.5625 -0.5625 ... -0.5625 -0.5625 -0.5625]
 [-0.5625 -0.4375 -0.4375 ... -0.9375 -0.9375 -0.5625]
 [-0.5625 -0.4375 -0.6875 ... -0.9375 -0.9375 -0.5625]
 ...
 [-0.5625 -0.9375 -0.9375 ... -0.4375 -0.0625 -0.5625]
 [-0.5625 -0.9375 -0.9375 ... -0.0625 -0.0625 -0.5625]
 [-0.5625 -0.5625 -0.5625 ... -0.5625 -0.5625 -0.5625]]


In [26]:
quant_signed_05_np(x[10])

array([[-0.5625, -0.5625, -0.5625, ..., -0.5625, -0.5625, -0.5625],
       [-0.5625, -0.4375, -0.4375, ..., -0.9375, -0.9375, -0.5625],
       [-0.5625, -0.4375, -0.6875, ..., -0.9375, -0.9375, -0.5625],
       ...,
       [-0.5625, -0.9375, -0.9375, ..., -0.4375, -0.0625, -0.5625],
       [-0.5625, -0.9375, -0.9375, ..., -0.0625, -0.0625, -0.5625],
       [-0.5625, -0.5625, -0.5625, ..., -0.5625, -0.5625, -0.5625]])

## Torch model

In [ ]:
# Evaluate with Torch model
correct = 0
randlist = random.sample(range(0, 10000), 256)
for i in randlist:    
    y = label_list[i]
    x = data_list[i].view(1,32,32)
    
    x = model.conv1(x)
    x = model.pool1(F.relu(x))
    #x = quant_signed_15(x)

    x = model.conv2(x)
    x = model.pool2(F.relu(x))
    #x = quant_signed_15(x)

    x = model.conv3(x)
    x = model.pool3(F.relu(x))
    #x = quant_signed_15(x)


    x = model.conv4(x)
    x = model.pool4(F.relu(x))
    #x = quant_signed_15(x)


    x = x.view(1,-1)
    x = model.fc5(x)
    #x = quant_signed_15(x)
    x = F.relu(x)

    x = model.fc6(x)
    #x = quant_signed_15(x)

    index_min1 = np.argmax(x.detach().numpy())
    if (index_min1 == y.item()):
        correct += 1
print(correct/len(randlist))

In [ ]:
len(data_list)

## Compar the result of two model

In [ ]:
# Start compare

# Custom model
i = 10
x = data_list[i].view(1,32,32).numpy()

x = padding(x)
x = conv_custom_fa(x,w1,b1)
x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa(x,w2,b2)
x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa(x,w3,b3)
x = maxpooling(x)
x = relu(x)

x = padding(x)
x = conv_custom_fa(x,w4,b4)
x = maxpooling(x)
x = relu(x)

x = x.reshape(1,1024)

x = fc_fa(x,w5,b5)

x1 = x

# Torch model

x = data_list[i].view(1,32,32)

x = model.conv1(x)
x = model.pool1(F.relu(x))

x = model.conv2(x)
x = model.pool2(F.relu(x))

x = model.conv3(x)
x = model.pool3(F.relu(x))


x = model.conv4(x)
x = model.pool4(F.relu(x))

x = x.view(1,-1)

x = model.fc5(x)

x2 = x

print(x1.shape, x2.shape)

if len(x1.shape) == 3:
    print("Conv Layer")
    a = x.shape[0]
    b = x.shape[1]
    c = x.shape[2]


    for i in range(a):
        for j in range(b):
            for k in range(c):
                if (x1[i][j][k] == x2[i][j][k].item()) == False:
                    print(x1[i][j][k] == x2[i][j][k].item())
                else:
                    print("Good")
elif len(x1.shape) == 2:
    print("FC Layer")
    a = x.shape[0]
    b = x.shape[1]

    for i in range(a):
        for j in range(b):
            if (x1[i][j] == x2[i][j].item()) == False:
                print(x1[i][j] ,x2[i][j].item())
            else:
                print("Good")